In [1]:
# OPTIMIZACION DEL BUTTERWORTH

import yfinance as yf
import pandas as pd 
import numpy as np
from scipy.signal import filtfilt, freqz, butter
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import plotly.express as px
from math import sin, cos, sqrt, pi
from scipy.optimize import newton


# FILTRO BUTTERWORTH Y BANDAS DE BOLLINGER

def BUTT(close,wc,order=1):

    B,A = butter(order,wc)
    y = []

    for n in range(len(close)):
        if n == 0:
            y.append(close[0])
        else:
            y_n = B[0]*(close[n] + close[n-1]) - A[1]*y[n-1]
            y.append(y_n)
    return y

def T_sma(wc):

    # Metodo de Newton para obtener el periodo de la SMA a partir de la frecuencia de corte (para las bollinger)
    # Cuando usamos una SMA es facil obtener las BB porque para la media de la std se usa el mismo periodo que la sma
    # Pero cuando estamos trabajando con la wc no sabemos el periodo. Una alternativa es programar las BB a partir de la wc
    wc = wc*pi
    func = lambda N: np.sin(wc*N/2) - (N/np.sqrt(2))*(np.sin(wc/2))
    deriv = lambda N: (wc/2)*np.cos(wc*N/2) - (1/np.sqrt(2))*np.sin(wc/2)
    N_0 = pi/wc  
    return int(np.round(newton(func, N_0, deriv)))

def Vol_L(close,wc,smooth,num_std = 2):
    return smooth - num_std*pd.Series(close).rolling(T_sma(wc)).std()

def Vol_U(close,wc,smooth,num_std = 2):
    return smooth + num_std*pd.Series(close).rolling(T_sma(wc)).std()

# La desviacion estandar se calcula usando una media movil pero es el periodo el que se define con el metodo de newton al no saber T

In [2]:
# SISTEMA 
class strat_BB(Strategy):

    wc = 0.04 # inicializamos a un valor arbitrario
    N = 2 # std dev from mean 
    sl_pct = 0.015

    def init(self):

        # Parameters 
        close = self.data.Close
        self.smooth = BUTT(close,self.wc)

        # Indicators 
        self.upper_band = self.I(Vol_U, close, self.wc, self.smooth, self.N)
        self.lower_band = self.I(Vol_L, close, self.wc, self.smooth, self.N)

    def next(self):

        price = self.data.Close[-1]

        # Buy   
        if crossover(self.lower_band,self.data.Close):
            self.position.close()
            self.buy(sl = price*(1-self.sl_pct))
            
        # Sell
        elif crossover(self.data.Close,self.upper_band):
            self.position.close()
            self.sell(sl = price*(1+self.sl_pct))

In [ ]:
# DATA IMPORT AND BACKTESTING 

xls = r'C:\Users\Usuario\Desktop\TFG\S&P 500 Companies (Standard and Poor 500).xlsx'
file_df = pd.read_excel(xls)
all_tickers = file_df['Symbol'][0:10]
tickers = all_tickers

mu = []; sigma = []; rets = []; ntrades = []

for i in range(len(tickers)):
    try:
        df = yf.download(tickers[i],  start = '2000-01-01', end = '2023-01-01', progress=False)
        iCash = 10000
        com = 0.00

        #BACKTESTING 
        bt = Backtest(df, strat_BB, cash = iCash, commission = com, exclusive_orders = True)

        # OUTPUTS
        output = bt.run()
        mu.append(output['Return (Ann.) [%]'])
        sigma.append(output['Volatility (Ann.) [%]'])
        ntrades.append(output['# Trades'])

    except: 
        rets.append(None); mu.append(None); sigma.append(None); ntrades.append(None)

# RESULTS
res = pd.DataFrame({'Ticker':tickers, 'Annualized Returns (%)':mu, 'Annualized Volatility (%)':sigma, 'Trades':ntrades}).dropna()
res = res[res['Trades'] != 0]
res = res[res['Annualized Volatility (%)'] < 100]
mean_values = res.mean()
mean_row = pd.DataFrame(mean_values).T
res = pd.concat([res, mean_row]).reset_index(drop=True)
res.loc[res.index[-1], 'Ticker'] = 'MEAN'

#bt.plot(plot_volume = False)
#res.to_excel(r'C:\Users\Usuario\Desktop\TFG\Resultados Excel\Segunda Simulacion\Comparativa SMA vs BUTT.xlsx', index = False)
res